In [1]:
%pip install cyvcf2 pyfaidx tqdm pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cyvcf2 import VCF
from pyfaidx import Fasta
from tqdm import tqdm
import pandas as pd
import random
import os
from collections import Counter, defaultdict

In [6]:
# Download VCF and reference 
import os, re, gzip, shutil, hashlib, requests
from pathlib import Path
from pyfaidx import Fasta

# ==== Setup base dirs ====
BASE = Path("./clinvar_data")
VCF_DIR = BASE / "vcf_GRCh38"
REF_DIR = BASE / "refs" / "GRCh38_gencode"

VCF_DIR.mkdir(parents=True, exist_ok=True)
REF_DIR.mkdir(parents=True, exist_ok=True)

# ==== Helper functions ====
def md5sum(path: Path) -> str:
    h = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1 << 20), b""):
            h.update(chunk)
    return h.hexdigest()

def download(url: str, dest: Path, chunk=1<<20):
    if dest.exists():
        print(f"Exists: {dest}")
        return
    print(f"Downloading {url} → {dest}")
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        with open(dest, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)

# ==== Part 1: ClinVar VCF (GRCh38) ====
clinvar_base = "https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/"

# Fetch directory listing and pick the newest clinvar_YYYYMMDD.vcf.gz
html = requests.get(clinvar_base, timeout=60).text
candidates = re.findall(r'href="(clinvar_(\d{8})\.vcf\.gz)"', html)
assert candidates, "No clinvar_YYYYMMDD.vcf.gz found at ClinVar FTP."
latest_name = sorted(candidates, key=lambda x: x[1])[-1][0]

vcf_url = clinvar_base + latest_name
tbi_url = vcf_url + ".tbi"
md5_url = vcf_url + ".md5"

vcf_path = VCF_DIR / latest_name
tbi_path = VCF_DIR / (latest_name + ".tbi")
md5_path = VCF_DIR / (latest_name + ".md5")

download(vcf_url, vcf_path)
download(tbi_url, tbi_path)
download(md5_url, md5_path)

# Verify MD5 (only if we just downloaded the VCF)
if vcf_path.exists() and md5_path.exists():
    with open(md5_path) as fh:
        expected = re.search(r'([0-9a-fA-F]{32})', fh.read()).group(1).lower()
    have = md5sum(vcf_path)
    if have == expected:
        print(f"MD5 ok for {vcf_path.name}")
    else:
        print(f"Warning: MD5 mismatch {have} != {expected}")

# ==== Part 2: Reference genome (GENCODE GRCh38 primary assembly) ====
gencode_base = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/latest_release/"
fasta_name = "GRCh38.primary_assembly.genome.fa.gz"
fasta_gz_url = gencode_base + fasta_name

fasta_gz_path = REF_DIR / fasta_name
fasta_path = REF_DIR / "GRCh38.primary_assembly.genome.fa"

download(fasta_gz_url, fasta_gz_path)

# Decompress only if .fa not already present
if not fasta_path.exists():
    print(f"Decompressing {fasta_gz_path} → {fasta_path}")
    with gzip.open(fasta_gz_path, "rb") as src, open(fasta_path, "wb") as dst:
        shutil.copyfileobj(src, dst)
else:
    print(f"Exists: {fasta_path}")

# Build .fai index if missing
if not Path(str(fasta_path) + ".fai").exists():
    print("Indexing FASTA with pyfaidx …")
    ref = Fasta(str(fasta_path))
    ref.close()
else:
    print(f"Exists: {fasta_path}.fai")

# ==== Summary ====
print("\nSummary:")
print(f"ClinVar VCF : {vcf_path}")
print(f"ClinVar TBI : {tbi_path}")
print(f"Reference FA: {fasta_path}")
print(f"FAI index   : {fasta_path}.fai")

MD5 ok for clinvar_20250928.vcf.gz
Exists: clinvar_data/refs/GRCh38_gencode/GRCh38.primary_assembly.genome.fa.gz
Exists: clinvar_data/refs/GRCh38_gencode/GRCh38.primary_assembly.genome.fa
Exists: clinvar_data/refs/GRCh38_gencode/GRCh38.primary_assembly.genome.fa.fai

Summary:
ClinVar VCF : clinvar_data/vcf_GRCh38/clinvar_20250928.vcf.gz
ClinVar TBI : clinvar_data/vcf_GRCh38/clinvar_20250928.vcf.gz.tbi
Reference FA: clinvar_data/refs/GRCh38_gencode/GRCh38.primary_assembly.genome.fa
FAI index   : clinvar_data/refs/GRCh38_gencode/GRCh38.primary_assembly.genome.fa.fai


In [10]:
# Define paths (Update these!)
VCF_PATH = "./clinvar_data/vcf_GRCh38/clinvar_20250928.vcf.gz"
REFERENCE_FASTA = "./clinvar_data/refs/GRCh38_gencode/GRCh38.primary_assembly.genome.fa"

In [11]:
ref_genome = Fasta(REFERENCE_FASTA) # load ref_genome

In [12]:
vcf = VCF(VCF_PATH) #load VCF

# Inspect data(VCF)

In [13]:
def inspect_clinvar_vcf(vcf_path, num_variants=5):
    """
    Specifically inspect ClinVar VCF structure with focus on CLNSIG field
    """
    vcf = VCF(vcf_path)
    
    print("CLINVAR VCF INSPECTION")
    print("=" * 50)
    
    # Show CLNSIG definition
    print("\nCLNSIG FIELD DEFINITION:")
    for line in vcf.raw_header.split('\n'):
        if 'CLNSIG' in line and line.startswith('##INFO'):
            print(line)
            break
    
    # Show variants with CLNSIG values
    print(f"\nFIRST {num_variants} VARIANTS WITH CLNSIG:")
    print("-" * 40)
    
    count = 0
    for variant in vcf:
        if count >= num_variants:
            break
            
        clnsig = variant.INFO.get('CLNSIG')
        if clnsig:  # Only show variants that have CLNSIG
            count += 1
            print(f"\nVariant {count}:")
            print(f"  {variant.CHROM}:{variant.POS} {variant.REF}>{','.join(variant.ALT)}")
            print(f"  CLNSIG: {clnsig}")
            print(f"  ID: {variant.ID}")
            
            # Show other relevant ClinVar fields
            for field in ['CLNREVSTAT', 'CLNDN', 'CLNDISDB']:
                value = variant.INFO.get(field)
                if value:
                    print(f"  {field}: {value}")
            print("-" * 30)
    
    vcf.close()


In [14]:
inspect_clinvar_vcf(VCF_PATH)

CLINVAR VCF INSPECTION

CLNSIG FIELD DEFINITION:
##INFO=<ID=CLNSIG,Number=.,Type=String,Description="Aggregate germline classification for this single variant; multiple values are separated by a vertical bar">

FIRST 5 VARIANTS WITH CLNSIG:
----------------------------------------

Variant 1:
  1:66926 AG>A
  CLNSIG: Uncertain_significance
  ID: 3385321
  CLNREVSTAT: criteria_provided,_single_submitter
  CLNDN: Retinitis_pigmentosa
  CLNDISDB: Human_Phenotype_Ontology:HP:0000547,MONDO:MONDO:0019200,MeSH:D012174,MedGen:C0035334,OMIM:268000,OMIM:PS268000,Orphanet:791
------------------------------

Variant 2:
  1:69134 A>G
  CLNSIG: Likely_benign
  ID: 2205837
  CLNREVSTAT: criteria_provided,_single_submitter
  CLNDN: not_specified
  CLNDISDB: MedGen:CN169374
------------------------------

Variant 3:
  1:69308 A>G
  CLNSIG: Uncertain_significance
  ID: 3925305
  CLNREVSTAT: criteria_provided,_single_submitter
  CLNDN: not_specified
  CLNDISDB: MedGen:CN169374
---------------------------

# count data

In [15]:
def count_clinvar_categories(vcf_path):
    """Count Benign, Pathogenic, and Uncertain variants"""
    vcf = VCF(vcf_path)
    
    benign = pathogenic = uncertain = other = 0
    total_with_clnsig = 0
    
    for variant in vcf:
        clnsig = variant.INFO.get('CLNSIG')
        if not clnsig:
            continue
            
        total_with_clnsig += 1
        
        if 'Benign' in clnsig or 'Likely_benign' in clnsig:
            benign += 1
        elif 'Pathogenic' in clnsig or 'Likely_pathogenic' in clnsig:
            pathogenic += 1
        elif 'Uncertain' in clnsig:
            uncertain += 1
        else:
            other += 1
    
    vcf.close()
    
    print("CLINVAR CATEGORY COUNTS")
    print("=" * 40)
    print(f"Benign/Likely_benign: {benign:,}")
    print(f"Pathogenic/Likely_pathogenic: {pathogenic:,}")
    print(f"Uncertain significance: {uncertain:,}")
    print(f"Other classifications: {other:,}")
    print(f"Total variants with CLNSIG: {total_with_clnsig:,}")
    
    return benign, pathogenic, uncertain, other

In [16]:
 count_clinvar_categories(VCF_PATH)

CLINVAR CATEGORY COUNTS
Benign/Likely_benign: 1,266,985
Pathogenic/Likely_pathogenic: 314,746
Uncertain significance: 1,940,705
Other classifications: 162,423
Total variants with CLNSIG: 3,684,859


(1266985, 314746, 1940705, 162423)

In [17]:
def count_clinvar_categories_detailed(vcf_path):
    """Count ClinVar categories with detailed breakdown"""
    vcf = VCF(vcf_path)
    
    # Detailed counters
    categories = {
        'Benign': 0,
        'Likely_benign': 0,
        'Pathogenic': 0,
        'Likely_pathogenic': 0,
        'Uncertain_significance': 0,
        'Conflicting_interpretations': 0,
        'Risk_factor': 0,
        'Drug_response': 0,
        'Affects': 0,
        'Protective': 0,
        'Association': 0,
        'Other': 0,
        'No_CLNSIG': 0
    }
    
    total_variants = 0
    total_with_clnsig = 0
    
    for variant in vcf:
        total_variants += 1
        clnsig = variant.INFO.get('CLNSIG')
        
        if not clnsig:
            categories['No_CLNSIG'] += 1
            continue
            
        total_with_clnsig += 1
        clnsig_str = str(clnsig)
        
        # Handle multiple classifications separated by |
        classifications = clnsig_str.split('|') if '|' in clnsig_str else [clnsig_str]
        
        found_category = False
        for classification in classifications:
            classification = classification.strip()
            
            if classification == 'Benign':
                categories['Benign'] += 1
                found_category = True
            elif classification == 'Likely_benign':
                categories['Likely_benign'] += 1
                found_category = True
            elif classification == 'Pathogenic':
                categories['Pathogenic'] += 1
                found_category = True
            elif classification == 'Likely_pathogenic':
                categories['Likely_pathogenic'] += 1
                found_category = True
            elif classification == 'Uncertain_significance':
                categories['Uncertain_significance'] += 1
                found_category = True
            elif 'Conflicting' in classification:
                categories['Conflicting_interpretations'] += 1
                found_category = True
            elif 'Risk_factor' in classification:
                categories['Risk_factor'] += 1
                found_category = True
            elif 'Drug_response' in classification:
                categories['Drug_response'] += 1
                found_category = True
            elif classification == 'Affects':
                categories['Affects'] += 1
                found_category = True
            elif classification == 'Protective':
                categories['Protective'] += 1
                found_category = True
            elif classification == 'Association':
                categories['Association'] += 1
                found_category = True
        
        if not found_category:
            categories['Other'] += 1
    
    vcf.close()
    
    # Print detailed breakdown
    print("DETAILED CLINVAR CATEGORY COUNTS")
    print("=" * 50)
    print(f"Total variants: {total_variants:,}")
    print(f"Variants with CLNSIG: {total_with_clnsig:,}")
    print(f"Variants without CLNSIG: {categories['No_CLNSIG']:,}")
    print()
    
    # Main categories
    print("MAIN CLASSIFICATIONS:")
    print("-" * 30)
    benign_total = categories['Benign'] + categories['Likely_benign']
    pathogenic_total = categories['Pathogenic'] + categories['Likely_pathogenic']
    
    print(f"Benign + Likely_benign: {benign_total:,}")
    print(f"  Benign: {categories['Benign']:,}")
    print(f"  Likely_benign: {categories['Likely_benign']:,}")
    
    print(f"Pathogenic + Likely_pathogenic: {pathogenic_total:,}")
    print(f"  Pathogenic: {categories['Pathogenic']:,}")
    print(f"  Likely_pathogenic: {categories['Likely_pathogenic']:,}")
    
    print(f"Uncertain_significance: {categories['Uncertain_significance']:,}")
    print()
    
    # Other categories
    print("OTHER CLASSIFICATIONS:")
    print("-" * 30)
    other_categories = ['Conflicting_interpretations', 'Risk_factor', 'Drug_response', 
                       'Affects', 'Protective', 'Association', 'Other']
    
    for category in other_categories:
        if categories[category] > 0:
            percentage = (categories[category] / total_with_clnsig) * 100
            print(f"{category}: {categories[category]:,} ({percentage:.1f}%)")
    
    return categories

# More advanced version that discovers all unique CLNSIG values automatically
def discover_all_clnsig_values(vcf_path, max_variants=50000):
    """Discover all unique CLNSIG values in the VCF"""
    vcf = VCF(vcf_path)
    
    unique_clnsig = set()
    clnsig_counter = Counter()
    
    for i, variant in enumerate(vcf):
        if i >= max_variants and max_variants > 0:
            break
            
        clnsig = variant.INFO.get('CLNSIG')
        if clnsig:
            # Handle multiple values
            if '|' in clnsig:
                for sig in clnsig.split('|'):
                    unique_clnsig.add(sig.strip())
                    clnsig_counter[sig.strip()] += 1
            else:
                unique_clnsig.add(clnsig.strip())
                clnsig_counter[clnsig.strip()] += 1
    
    vcf.close()
    
    print("ALL UNIQUE CLNSIG VALUES FOUND:")
    print("=" * 40)
    for clnsig in sorted(unique_clnsig):
        count = clnsig_counter[clnsig]
        print(f"'{clnsig}': {count:,} variants")
    
    return unique_clnsig, clnsig_counter

# Simple version for just the main categories you asked for
def count_main_categories_separate(vcf_path):
    """Count main ClinVar categories separately"""
    vcf = VCF(vcf_path)
    
    counts = {
        'Benign': 0,
        'Likely_benign': 0,
        'Pathogenic': 0,
        'Likely_pathogenic': 0,
        'Uncertain_significance': 0,
        'Other': 0,
        'No_CLNSIG': 0
    }
    
    total_variants = 0
    
    for variant in vcf:
        total_variants += 1
        clnsig = variant.INFO.get('CLNSIG')
        
        if not clnsig:
            counts['No_CLNSIG'] += 1
            continue
        
        clnsig_str = str(clnsig)
        classifications = clnsig_str.split('|') if '|' in clnsig_str else [clnsig_str]
        
        found = False
        for classification in classifications:
            classification = classification.strip()
            if classification in counts:
                counts[classification] += 1
                found = True
                break  # Take the first matching classification
        
        if not found:
            counts['Other'] += 1
    
    vcf.close()
    
    print("MAIN CLINVAR CATEGORIES (SEPARATE)")
    print("=" * 40)
    print(f"Total variants: {total_variants:,}")
    print(f"With CLNSIG: {total_variants - counts['No_CLNSIG']:,}")
    print()
    
    for category in ['Benign', 'Likely_benign', 'Pathogenic', 'Likely_pathogenic', 'Uncertain_significance']:
        count = counts[category]
        percentage = (count / total_variants) * 100
        print(f"{category}: {count:,} ({percentage:.1f}%)")
    
    if counts['Other'] > 0:
        print(f"Other classifications: {counts['Other']:,}")
    
    print(f"No CLNSIG: {counts['No_CLNSIG']:,}")
    
    return counts


In [18]:
# Usage examples:
print("=== DETAILED BREAKDOWN ===")
detailed_counts = count_clinvar_categories_detailed(VCF_PATH)



=== DETAILED BREAKDOWN ===
DETAILED CLINVAR CATEGORY COUNTS
Total variants: 3,689,385
Variants with CLNSIG: 3,684,859
Variants without CLNSIG: 4,526

MAIN CLASSIFICATIONS:
------------------------------
Benign + Likely_benign: 1,206,906
  Benign: 209,694
  Likely_benign: 997,212
Pathogenic + Likely_pathogenic: 280,717
  Pathogenic: 175,304
  Likely_pathogenic: 105,413
Uncertain_significance: 1,940,523

OTHER CLASSIFICATIONS:
------------------------------
Conflicting_interpretations: 150,101 (4.1%)
Affects: 146 (0.0%)
Other: 106,481 (2.9%)


In [19]:
print("\n=== MAIN CATEGORIES SEPARATE ===")
main_counts = count_main_categories_separate(VCF_PATH)




=== MAIN CATEGORIES SEPARATE ===
MAIN CLINVAR CATEGORIES (SEPARATE)
Total variants: 3,689,385
With CLNSIG: 3,684,859

Benign: 209,694 (5.7%)
Likely_benign: 997,212 (27.0%)
Pathogenic: 175,304 (4.8%)
Likely_pathogenic: 105,413 (2.9%)
Uncertain_significance: 1,940,523 (52.6%)
Other classifications: 256,713
No CLNSIG: 4,526


In [20]:
print("\n=== DISCOVER ALL VALUES ===")
unique_values, counter = discover_all_clnsig_values(VCF_PATH, max_variants=10000)


=== DISCOVER ALL VALUES ===
ALL UNIQUE CLNSIG VALUES FOUND:
'Benign': 474 variants
'Benign/Likely_benign': 117 variants
'Conflicting_classifications_of_pathogenicity': 213 variants
'Likely_benign': 3,073 variants
'Likely_pathogenic': 106 variants
'Pathogenic': 131 variants
'Pathogenic/Likely_pathogenic': 29 variants
'Uncertain_significance': 5,850 variants
'no_classifications_from_unflagged_records': 1 variants
'not_provided': 5 variants
'risk_factor': 1 variants


In [21]:
def check_vcf_chromosomes(vcf_path):
    """Check what chromosome naming convention the VCF uses"""
    vcf = VCF(vcf_path)
    
    print("VCF CHROMOSOME NAMING")
    print("=" * 40)
    
    chrom_counter = {}
    
    for i, variant in enumerate(vcf):
        chrom = variant.CHROM
        chrom_counter[chrom] = chrom_counter.get(chrom, 0) + 1
        
        if i >= 100:  # Check first 100 variants
            break
    
    vcf.close()
    
    print("Chromosomes found in VCF (first 100 variants):")
    for chrom, count in chrom_counter.items():
        print(f"  '{chrom}': {count} variants")
    
    return list(chrom_counter.keys())

In [22]:
vcf.seqnames

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 'X',
 'Y',
 'MT',
 'NT_113889.1',
 'NT_187633.1',
 'NT_187661.1',
 'NT_187693.1',
 'NW_009646201.1']

In [23]:
vcf_chroms = check_vcf_chromosomes(VCF_PATH)

VCF CHROMOSOME NAMING
Chromosomes found in VCF (first 100 variants):
  '1': 101 variants


# Inspect data REF GENOME

In [24]:
# Basic information
print("REFERENCE GENOME INFO:")
print("=" * 40)
print(f"Number of chromosomes/contigs: {len(ref_genome.keys())}")
print(f"Chromosome names: {list(ref_genome.keys())[:10]}...")  # First 10

# Look at specific chromosomes
print("\nCHROMOSOME LENGTHS:")
print("-" * 30)
for chrom in ['chr1', 'chr2', 'chr3', 'chrX', 'chrY']:
    if chrom in ref_genome:
        print(f"Chromosome {chrom}: {len(ref_genome[chrom]):,} bp")

# Examine sequence content
print("\nSAMPLE SEQUENCE (first 100 bp of chr1):")
print("-" * 50)
if 'chr10' in ref_genome: #see sample chromosome
    sample_seq = ref_genome['chr10'][50000:50100]
    print(sample_seq)


REFERENCE GENOME INFO:
Number of chromosomes/contigs: 194
Chromosome names: ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10']...

CHROMOSOME LENGTHS:
------------------------------
Chromosome chr1: 248,956,422 bp
Chromosome chr2: 242,193,529 bp
Chromosome chr3: 198,295,559 bp
Chromosome chrX: 156,040,895 bp
Chromosome chrY: 57,227,415 bp

SAMPLE SEQUENCE (first 100 bp of chr1):
--------------------------------------------------
GGGGAATGCCTCGATCTAACCAGTGAAGGTGTCAGTAAGCATTAGCAAATATTTGAATCTCCTGCAGAAAGGCTTCCGGGTAAATTAGAGTGGCCAGTTC


In [25]:
vcf

# Link ref genome with VCF

In [26]:
def debug_vcf_alt_issues(vcf_path, max_variants=5000):
    """Debug ALT field issues in the VCF"""
    vcf = VCF(vcf_path)
    
    print("DEBUGGING VCF ALT FIELDS:")
    print("=" * 40)
    
    alt_issues = 0
    for i, variant in enumerate(vcf):
        if i >= max_variants:
            break
            
        if not variant.ALT or len(variant.ALT) == 0:
            alt_issues += 1
            print(f"Variant with no ALT: {variant.CHROM}:{variant.POS}")
            print(f"  REF: '{variant.REF}'")
            print(f"  ALT: {variant.ALT}")
            print(f"  ID: {variant.ID}")
            print(f"  FILTER: {variant.FILTER}")
            print()
    
    vcf.close()
    
    if alt_issues == 0:
        print("No ALT issues found in first {max_variants} variants")
    else:
        print(f"Found {alt_issues} variants with ALT issues")

In [27]:
debug_vcf_alt_issues(VCF_PATH) # some varinat in ALT not provided

DEBUGGING VCF ALT FIELDS:
Variant with no ALT: 1:942451
  REF: 'T'
  ALT: []
  ID: 836156
  FILTER: None

Found 1 variants with ALT issues


In [28]:
def get_variant_sequences_robust(vcf_path, ref_genome, window_size=10, max_variants=1000):
    
    variant_sequences = []
    stats = {
        'total_processed': 0,
        'no_alt': 0,
        'non_snv': 0,
        'chrom_not_found': 0,
        'position_oob': 0,
        'ref_mismatch': 0,
        'other_errors': 0
    }
    
    for i, variant in enumerate(vcf):
        if i >= max_variants:
            break
            
        stats['total_processed'] += 1
        
        try:
            # Check ALT alleles
            if not variant.ALT:
                stats['no_alt'] += 1
                continue
                
            # Check if ALT is a list with actual values
            if len(variant.ALT) == 0 or variant.ALT[0] is None:
                stats['no_alt'] += 1
                continue
            
            # Only process SNVs
            if len(variant.REF) != 1:
                stats['non_snv'] += 1
                continue
                
            if any(alt is None or len(alt) != 1 for alt in variant.ALT):
                stats['non_snv'] += 1
                continue
            
            # Map chromosome
            vcf_chrom = variant.CHROM
            ref_chrom = f'chr{vcf_chrom}' if vcf_chrom != 'MT' else 'chrM'
            
            if ref_chrom not in ref_genome:
                stats['chrom_not_found'] += 1
                continue
            
            # Check position
            pos = variant.POS - 1
            if pos < 0 or pos >= len(ref_genome[ref_chrom]):
                stats['position_oob'] += 1
                continue
            
            # Get reference base and verify
            ref_base = ref_genome[ref_chrom][pos:pos+1].seq
            if ref_base != variant.REF:
                stats['ref_mismatch'] += 1
                continue
            
            # Extract sequence
            start = max(0, pos - window_size)
            end = min(len(ref_genome[ref_chrom]), pos + window_size + 1)
            ref_sequence = ref_genome[ref_chrom][start:end].seq
            
            variant_data = {
                'chrom': vcf_chrom,
                'pos': variant.POS,
                'ref': variant.REF,
                'alt': variant.ALT[0],
                'clnsig': variant.INFO.get('CLNSIG'),
                'ref_sequence': ref_sequence,
                'variant_position': pos - start,
                'window_start': start + 1,
                'window_end': end
            }
            
            variant_sequences.append(variant_data)
            
        except Exception as e:
            stats['other_errors'] += 1
            if stats['other_errors'] <= 3:
                print(f"Unexpected error at {variant.CHROM}:{variant.POS}: {e}")
            continue
    
    vcf.close()
    
    # Print statistics
    print("PROCESSING STATISTICS:")
    print(f"Total variants processed: {stats['total_processed']}")
    print(f"Successfully extracted: {len(variant_sequences)}")
    print(f"No ALT alleles: {stats['no_alt']}")
    print(f"Non-SNV variants: {stats['non_snv']}")
    print(f"Chromosome not found: {stats['chrom_not_found']}")
    print(f"Position out of bounds: {stats['position_oob']}")
    print(f"Reference mismatches: {stats['ref_mismatch']}")
    print(f"Other errors: {stats['other_errors']}")
    
    return variant_sequences



In [29]:

sequences = get_variant_sequences_robust(VCF_PATH, ref_genome)

PROCESSING STATISTICS:
Total variants processed: 1000
Successfully extracted: 922
No ALT alleles: 1
Non-SNV variants: 77
Chromosome not found: 0
Position out of bounds: 0
Reference mismatches: 0
Other errors: 0


In [30]:
sequences[:2]

[{'chrom': '1',
  'pos': 69134,
  'ref': 'A',
  'alt': 'G',
  'clnsig': 'Likely_benign',
  'ref_sequence': 'GATTCTCAGGAACTCCAGACC',
  'variant_position': 10,
  'window_start': 69124,
  'window_end': 69144},
 {'chrom': '1',
  'pos': 69308,
  'ref': 'A',
  'alt': 'G',
  'clnsig': 'Uncertain_significance',
  'ref_sequence': 'ACAGCCCCCAAGATGATTACT',
  'variant_position': 10,
  'window_start': 69298,
  'window_end': 69318}]